In [1]:
import turicreate as tc
from turicreate import SFrameBuilder
import os
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
from pathlib import Path
import random

In [2]:
dataDir='/Volumes/Data/cocodata'
dataVal='val2017'
annVal='{}/annotations/instances_{}.json'.format(dataDir,dataVal)
dataTrain='train2017'
annTrain='{}/annotations/instances_{}.json'.format(dataDir,dataTrain)

In [3]:
# initialize COCO api for instance annotations
cocoVal=COCO(annVal)
cocoTrain=COCO(annTrain)

loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
loading annotations into memory...
Done (t=22.58s)
creating index...
index created!


In [4]:
catsVal = cocoVal.loadCats(cocoVal.getCatIds())
catsTrain = cocoTrain.loadCats(cocoTrain.getCatIds())

In [5]:
def getCatName(catID):
    which = next(i for i,x in enumerate(catsTrain) if x['id']==catID)
    name = catsTrain[which]["name"]
    return name



In [6]:

nms=[cat['name'] for cat in catsVal]
print('COCO categories: \n{}\n'.format(' '.join(nms)))
nms = set([cat['supercategory'] for cat in catsVal])
print('COCO supercategories: \n{}'.format(' '.join(nms)))


COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
animal food accessory kitchen furniture person appliance vehicle electronic outdoor sports indoor


In [7]:
# get all images containing given categories, select one at random
imageIdSet=set()
catNms=['person','dog','cat']
catIds = cocoTrain.getCatIds(catNms=catNms);
imgIds = cocoTrain.getImgIds(catIds=catIds);
#print(catIds, imgIds)
#imgIds = coco.getImgIds(imgIds = [324158])
#img = cocoTrain.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
for catId in catIds:
    thisImgIds = cocoTrain.getImgIds(catIds=catId)
    random.shuffle(thisImgIds)
    thisImgIds = thisImgIds[:4000]
    #print(len(thisImgIds))
    imageIdSet.update(thisImgIds)
print(len(imageIdSet), len(imgIds))

11662 49


In [8]:
sb = SFrameBuilder([tc.Image,list],column_names=['image','annotations'])
import ipywidgets as widgets
from IPython.display import display
from decimal import Decimal
out = widgets.Output(layout={'border': '1px solid black'})
twoinplaces = Decimal('0.01')
display(out)

for index,imageId in enumerate(imageIdSet):
    img = cocoTrain.loadImgs([imageId])
    imgName = img[0]['file_name']
    imgPath = '%s/images/%s/%s'%(dataDir,dataTrain,imgName)
    out.clear_output(wait=True)
    with out:
        #image.show()
        print(f'process image number: {imgName}, progress: {Decimal((index + 1) / len(imageIdSet) * 100).quantize(twoinplaces)}%')
    image=tc.Image(imgPath)
    annIds = cocoTrain.getAnnIds(imgIds=[imageId])
    anns = cocoTrain.loadAnns(annIds)
    annotations=[]
    for ann in anns:
        thisCatId = ann["category_id"]
        if thisCatId in catIds:
            name = getCatName(thisCatId)
            bbox = ann['bbox']
            #print(f' bounding box: {bbox}, this cat: {thisCatId}')
            #coco use top/left coordinate, turicreate needs center
            bboxDict = {'width': bbox[2], 'height': bbox[3], 'x':bbox[0] + bbox[2] / 2, 'y':bbox[1] + bbox[3] / 2}
            annotation={'label':name, 'coordinates': bboxDict}
            annotations.append(annotation)
    sb.append([image,annotations])
    
sf = sb.close() 
#sf['image_with_ground_truth'] = \
#    tc.object_detector.util.draw_bounding_boxes(sf['image'], sf['annotations'])
#sf.explore()

Output(layout=Layout(border='1px solid black'))

In [9]:
sf.save('/Volumes/Data/yoloTiny.sframe')
## sframe above is for training

In [3]:
#sf['image_with_ground_truth'] = \
#    tc.object_detector.util.draw_bounding_boxes(sf['image'], sf['annotations'])
#sf.explore()

Materializing SFrame

In [7]:
#validation
imageIdSet=set()
catNms=['person','cat','dog']
catIds = cocoVal.getCatIds(catNms=catNms);
imgIds = cocoVal.getImgIds(catIds=catIds);
#print(catIds, imgIds)
#imgIds = coco.getImgIds(imgIds = [324158])
#img = cocoTrain.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
for catId in catIds:
    thisImgIds = cocoVal.getImgIds(catIds=catId)
    random.shuffle(thisImgIds)
    thisImgIds = thisImgIds[:500]
    print(len(thisImgIds))
    imageIdSet.update(thisImgIds)

500
184
177


In [8]:
sb_val = SFrameBuilder([tc.Image,list],column_names=['image','annotations'])
import ipywidgets as widgets
from IPython.display import display
from decimal import Decimal
out = widgets.Output(layout={'border': '1px solid black'})
twoinplaces = Decimal('0.01')
display(out)

for index,imageId in enumerate(imageIdSet):
    img = cocoVal.loadImgs([imageId])
    imgName = img[0]['file_name']
    imgPath = '%s/images/%s/%s'%(dataDir,dataVal,imgName)
    out.clear_output(wait=True)
    with out:
        #image.show()
        print(f'process image number: {imgName}, progress: {Decimal((index + 1) / len(imageIdSet) * 100).quantize(twoinplaces)}%')
    image=tc.Image(imgPath)
    annIds = cocoTrain.getAnnIds(imgIds=[imageId])
    anns = cocoTrain.loadAnns(annIds)
    annotations=[]
    for ann in anns:
        thisCatId = ann["category_id"]
        if thisCatId in catIds:
            name = getCatName(thisCatId)
            bbox = ann['bbox']
            #print(f' bounding box: {bbox}, this cat: {thisCatId}')
            #coco use top/left coordinate, turicreate needs center
            bboxDict = {'width': bbox[2], 'height': bbox[3], 'x':bbox[0] + bbox[2] / 2, 'y':bbox[1] + bbox[3] / 2}
            annotation={'label':name, 'coordinates': bboxDict}
            annotations.append(annotation)
    sb_val.append([image,annotations])
    
sf = sb_val.close() 

Output(layout=Layout(border='1px solid black'))

In [9]:
sf.save('/Volumes/Data/yoloVal.sframe')